# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pimentel,-6.84,-79.93,66.20,29,75,5.82,PE,1585764415
1,1,victoria,22.29,114.16,69.01,88,40,18.34,HK,1585764288
2,2,padang,-0.95,100.35,79.90,75,99,1.63,ID,1585764115
3,3,hasaki,35.73,140.83,55.00,100,75,4.70,JP,1585764415
4,4,kahului,20.89,-156.47,70.00,88,1,7.63,US,1585764415


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [5]:
#layer for heatmap
locations = cities_df[["Lat", "Lng"]]

humidity = cities_df["Humidity"]

fig = gmaps.figure()
hm_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(hm_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#creating new df with cleaned data
new_df = pd.DataFrame(cities_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

#adjusting features; max temp between 70 & 80 degrees; wind speed less than 10; and zero cloudiness
max_temp = (new_df["Max Temp"] <= 80) & (new_df["Max Temp"] > 70)
wind_speed = new_df["Wind Speed"] < 10
cloudiness = new_df["Cloudiness"] == 0

# new df with adjusted features
new_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
81,sakit,78.76,9.51,0
103,ormara,76.39,9.46,0
215,tura,72.75,3.09,0
274,chabahar,75.97,8.39,0
308,myitkyina,72.43,2.64,0
422,ghotki,77.00,5.70,0
484,nalut,73.78,6.08,0
544,sabha,78.24,9.01,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#create new dataframe
hotel_df = pd.DataFrame(cities_df, columns=["City", "Country", "Lat", "Lng"])

#adding "hotel name" column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,pimentel,PE,-6.84,-79.93,
1,victoria,HK,22.29,114.16,
2,padang,ID,-0.95,100.35,
3,hasaki,JP,35.73,140.83,
4,kahului,US,20.89,-156.47,
...,...,...,...,...,...
545,ingham,US,42.60,-84.38,
546,sakaraha,MG,-22.90,44.53,
547,port blair,IN,11.67,92.75,
548,fort nelson,CA,58.81,-122.70,


In [12]:
# search parameters for hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# for loop thru df to find hotels
for index, row in hotel_df.iterrows():
    # get lat and lng from hotel dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # making request; get url for ones that meet params
    hotel_names = requests.get(base_url, params=params)
    
    # jsonify
    hotel_names = hotel_names.json()
    
    # store hotel name; skip if not found
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_names["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


KeyboardInterrupt: 

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display map
fig

Figure(layout=FigureLayout(height='420px'))